# Classifying Heart Disease Using Logistic Regression

## 1. Introduction

In this project, we'll be exploring the [Heart Disease](https://archive.ics.uci.edu/dataset/45/heart+disease) dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/). This dataset originates from the Cleveland Clinic Foundation, which recorded various patient characteristics, including age and chest pain, to classify the presence of heart disease in individuals. This is a prime example of how we can use machine learning to solve problems that have a real impact on people's lives.

We'll walk through the machine learning pipeline, starting from examining the dataset itself to creating a logistic regression model. Specifically, we'll use a dataset that [Dataquest](https://www.dataquest.io/) partially cleaned to facilitate binary classification. The original dataset, which contains multiple classes, can be downloaded from the original website. Moreover, the target variable in the original dataset is named `num`.

To get started, let's import the relevant libraries and the dataset we'll be using.

In [1]:
# Import the relevant libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Load the heart disease dataset and display the first few rows
heart = pd.read_csv("Datasets/heart_disease.csv")
heart.head()

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,present
0,1,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,2,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,1
2,3,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,4,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,5,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


The first few rows show patients with varying levels of chest pain, blood pressure, and cholesterol. Fasting blood sugar and exercise-induced angina differ across patients. The target variable, `present`, indicates whether heart disease is diagnosed. In addition, the features include measurements like maximum heart rate, ST depression, and the number of major vessels, which vary across individuals.

## 2. Exploring and Cleaning the Dataset

Before we build our classification model, we should further explore the dataset and make any necessary adjustments. This may include converting categorical variables into dummy variables or scaling features.

In [2]:
# Display summary information about the dataset
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  303 non-null    int64  
 1   age         303 non-null    int64  
 2   sex         303 non-null    int64  
 3   cp          303 non-null    int64  
 4   trestbps    303 non-null    int64  
 5   chol        303 non-null    int64  
 6   fbs         303 non-null    int64  
 7   restecg     303 non-null    int64  
 8   thalach     303 non-null    int64  
 9   exang       303 non-null    int64  
 10  oldpeak     303 non-null    float64
 11  slope       303 non-null    int64  
 12  ca          303 non-null    object 
 13  thal        303 non-null    object 
 14  present     303 non-null    int64  
dtypes: float64(1), int64(12), object(2)
memory usage: 35.6+ KB


We can consult the official [page](https://archive.ics.uci.edu/dataset/45/heart+disease) to learn more about the column names, most of which include descriptions.

The dataset contains `303` rows and `15` columns. The `present` column is our binary outcome of interest, where `0` indicates the absence of heart disease, and `1` indicates its presence. Surprisingly, the columns `ca` and `thal` are of object data type instead of numerical, so let's examine them further.

In [3]:
# Display the count of each unique value in 'thal' and 'ca' columns
display(heart['thal'].value_counts())
display(heart['ca'].value_counts())

thal
3.0    166
7.0    117
6.0     18
?        2
Name: count, dtype: int64

ca
0.0    176
1.0     65
2.0     38
3.0     20
?        4
Name: count, dtype: int64

The `?` values appear to indicate missing or unknown data. Since `?` in the `thal` and `ca` columns occurred `2` and `4` times, respectively, we can replace `?` with the most common value in each column.

Since the `Unnamed: 0` column appears to be an index column, it doesn't contain any meaningful information for analysis, so it should be removed to avoid potential confusion.

In [4]:
# Replace '?' in the 'thal' column with the most common value and convert to float
heart['thal'] = heart['thal'].replace('?', heart['thal'].mode()[0])
heart['thal'] = heart['thal'].astype(float)

# Replace '?' in the 'ca' column with the most common value and convert to float
heart['ca'] = heart['ca'].replace('?', heart['ca'].mode()[0])
heart['ca'] = heart['ca'].astype(float)

# Remove the index column 'Unnamed: 0'
heart.drop('Unnamed: 0', axis=1, inplace=True)

Next, let's compute the proportion of each class in the target variable `present` to check for class imbalance.

In [5]:
# Calculate the proportion of each class in the target variable 'present'
heart['present'].value_counts(normalize=True)

present
0    0.541254
1    0.458746
Name: proportion, dtype: float64

The dataset is slightly imbalanced, with `54.12%` of the instances indicating no heart disease and `45.87%` indicating the presence of heart disease. 

Next, we'll check the mean of the predictors based on the outcome, as they could be informative for classification.

In [6]:
# Group by 'present' and calculate the mean for each column, rounded to 2 decimal places
by_disease_status = heart.groupby(heart['present']).mean()
by_disease_status.round(2)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
present,,,,,,,,,,,,,
0,52.59,0.56,2.79,129.25,242.64,0.14,0.84,158.38,0.14,0.59,1.41,0.27,3.79
1,56.63,0.82,3.59,134.57,251.47,0.16,1.17,139.26,0.55,1.57,1.83,1.13,5.82


As we can see, there are noticeable differences in the mean of each column based on the outcome, with all columns except `thalach` showing a larger mean in the presence of heart disease. Therefore, we will keep all features to create our logistic regression model.

## 3. Dividing the Dataset

Now that we have selected our predictors, we need to set aside data for final model assessment. We’ll need the following:
1. A training set to estimate the logistic regression coefficients.
2. A test set to evaluate the model’s predictive ability.

The model will be trained on the training set, and its predictive performance will be assessed on the test set. Additionally, we need to ensure that both sets contain both **cases** and **non-cases**.

In [7]:
# Select predictors and target variable
X = heart.drop(["present"], axis=1)
y = heart["present"]

# Split the data into training and test sets (70% training, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

# Initialize StandardScaler and transform the training and test sets
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Print the number of cases and non-cases in the training set
print("Y_train (no disease):", sum(y_train == 0))
print("Y_train (disease):", sum(y_train == 1))

# Print the number of cases and non-cases in the test set
print("Y_test (no disease):", sum(y_test == 0))
print("Y_test (disease):", sum(y_test == 1))

Y_train (no disease): 115
Y_train (disease): 97
Y_test (no disease): 49
Y_test (disease): 42


The distribution of cases and non-cases is fairly balanced in both the training and test sets, with slightly more non-disease cases than disease cases in each. This suggests that the model will have a reasonable opportunity to learn from both classes during training and be evaluated on them in the test set.

## 4. Building the Model

With the heart disease dataset divided, let’s build the classification model and conduct some initial assessments. Here are some guiding questions to consider:
- What is the training accuracy, sensitivity, and specificity?
- Does the model perform better on cases or non-cases, or does it perform equally well?

Typically, training metrics are optimistic estimations of the model's performance, so we should expect slightly poorer metrics when the model is applied to new data. If the training metrics are too high, it might indicate that our model is overfitted.

In [8]:
# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Calculate the training accuracy, and make predictions on the training data
accuracy = model.score(X_train, y_train)
predictions = model.predict(X_train)

# Calculate true positives, false positives, true negatives, and false negatives
tp = sum((predictions == 1) & (y_train == 1))
fp = sum((predictions == 1) & (y_train == 0))
tn = sum((predictions == 0) & (y_train == 0))
fn = sum((predictions == 0) & (y_train == 1))

# Compute sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Print training metrics: accuracy, sensitivity, and specificity
print("Training Accuracy:", round(accuracy, 4))
print("Training Sensitivity:", round(sensitivity, 4))
print("Training Specificity:", round(specificity, 4))

Training Accuracy: 0.8585
Training Sensitivity: 0.8041
Training Specificity: 0.9043


The model shows a strong training accuracy of `85.85%`, with higher specificity (`90.43%`) compared to sensitivity (`80.41%`). This indicates that the model is better at correctly identifying non-disease cases (`specificity`) than disease cases (`sensitivity`). The lower sensitivity suggests that the model might miss some cases of heart disease.

## 5. Interpreting the Model Coefficients

Now that we've created our model, let's examine the coefficients to see if they make sense given the problem. Logistic regression relates the binary outcome to the linear combination of predictors via the link function: 

$$\log\left(\frac{E[Y]}{1 - E[Y]}\right) = \beta_0 + \beta_1 X$$

The predictors affect the outcome on the log-odds scale, and the non-intercept coefficients represent the log-odds ratio for a unit increase in a predictor: 
$$\log\left(\frac{O_1}{O_0}\right) = \beta_1$$

`𝑶₀` represents the odds ratio when the predictor is `0`, and `𝑶₁` represents the odds ratio when the predictor is `1`. Usually, we're interested in examining these effects on the odds scale, so we exponentiate both sides to get the following: 

$$\boldsymbol{O_1 = e^{\beta_1} O_0}$$

Let’s examine what our predictors suggest about their relationship with heart disease.

In [9]:
# Define the list of predictor names
predictors = heart.columns

# Print the coefficients for each predictor
print("Coefficients for each predictor:")
for pred, coef in zip(predictors, model.coef_[0]):
    print(pred, ":", round(coef, 2))

print()

# Print the exponentiated coefficients (odds ratios) for each predictor
print("Exponentiated coefficients for each predictor:")
for pred, coef in zip(predictors, model.coef_[0]):
    print(pred, ":", round(np.exp(coef), 2))

Coefficients for each predictor:
age : -0.06
sex : 0.7
cp : 0.48
trestbps : 0.24
chol : 0.23
fbs : -0.12
restecg : 0.16
thalach : -0.44
exang : 0.51
oldpeak : 0.44
slope : 0.2
ca : 0.97
thal : 0.43

Exponentiated coefficients for each predictor:
age : 0.94
sex : 2.02
cp : 1.62
trestbps : 1.27
chol : 1.25
fbs : 0.89
restecg : 1.18
thalach : 0.64
exang : 1.66
oldpeak : 1.55
slope : 1.22
ca : 2.65
thal : 1.54


The coefficients suggest the following insights while holding the other predictors constant:
- `ca` and `sex` have the highest positive coefficients, indicating that more vessels colored and male sex are strongly associated with increased odds of heart disease.
- Most other predictors (`cp`, `trestbps`, `chol`, `restecg`, `exang`, `oldpeak`, `slope`, and `thal`) also show positive coefficients, with varying strengths, indicating a general association with higher odds of heart disease.
- `age`, `fbs`, and `thalach` have negative coefficients, suggesting that increased age, fasting blood sugar, or maximum heart rate are associated with decreased odds of heart disease.

The exponentiated coefficients show that each unit increase in these predictors changes the odds of heart disease by the given multiplier. For example, a one-unit increase in the number of vessels colored (`ca`) increases the odds of heart disease by about `2.65` times.

## 6. Final Model Evaluation

Finally, we can assess the predictive ability of our logistic regression model using the test set.

In [10]:
# Calculate the accuracy of the model on the test set and make predictions
accuracy = model.score(X_test, y_test)
predictions = model.predict(X_test)

# Calculate true positives, false positives, true negatives, and false negatives
tp = sum((predictions == 1) & (y_test == 1))
fp = sum((predictions == 1) & (y_test == 0))
tn = sum((predictions == 0) & (y_test == 0))
fn = sum((predictions == 0) & (y_test == 1))

# Compute sensitivity and specificity based on the test set
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Print test metrics: accuracy, sensitivity, and specificity
print("Test Accuracy:", round(accuracy, 4))
print("Test Sensitivity:", round(sensitivity, 4))
print("Test Specificity:", round(specificity, 4))

Test Accuracy: 0.8681
Test Sensitivity: 0.881
Test Specificity: 0.8571


The test metrics show that the model performs slightly better on the test set than on the training set. Specifically, test accuracy increased to `86.81%` from `85.85%` in training, indicating a slight improvement in overall predictive performance. However, specificity decreased from `90.43%` to `85.71%`, suggesting the model is less effective at identifying non-cases of heart disease in the test set. Conversely, sensitivity rose significantly from `80.41%` to `88.1%`, showing a major improvement in correctly identifying cases. Overall, these results suggest that the model generalizes well to new data without overfitting.

## 7. Conclusion

In this project, we explored a dataset originating from the Cleveland Clinic Foundation, which recorded various patient characteristics to classify the presence of heart disease. Specifically, we used a dataset that [Dataquest](https://www.dataquest.io/) partially cleaned to facilitate binary classification, with the target variable changed from `num` (in the original dataset) to `present`.

The dataset initially contained `303` rows and `15` columns, with the `present` column serving as our binary outcome of interest, where `0` and `1` indicate the absence and presence of heart disease, respectively. Additionally, we replaced the `?` values in the `thal` and `ca` columns, which indicate missing or unknown data, with the most common value in each column. We also removed the `Unnamed: 0` column, as it appeared to be an index column and didn't contain any meaningful information for analysis.

The dataset is slightly imbalanced, with `54.12%` of the instances indicating no heart disease and `45.87%` indicating the presence of heart disease. After examining the mean of each column based on the outcome, we noticed that all columns except `thalach` show a larger mean in the presence of heart disease. Also, we decided to keep all features to create our logistic regression model. Next, we split the data into a training set to estimate the coefficients and a test set to evaluate the model’s predictive ability.

With the dataset divided, we built our logistic regression model and conducted some initial assessments. The classification model achieved a strong training accuracy of `85.85%`, with higher specificity (`90.43%`) compared to sensitivity (`80.41%`). This indicates that the model is better at correctly identifying non-disease cases (`specificity`) than disease cases (`sensitivity`). Additionally, the lower sensitivity suggests that the model might miss some cases of heart disease.

The coefficients suggest the following while holding the other predictors constant: `ca` and `sex` have the highest positive coefficients, indicating that more vessels colored and male sex are strongly associated with increased odds of heart disease. Most other predictors also show positive coefficients, with varying strengths, indicating a general association with higher odds of heart disease. The columns `age`, `fbs`, and `thalach` have negative coefficients, suggesting that increased age, fasting blood sugar, or maximum heart rate are associated with decreased odds of heart disease. Furthermore, the exponentiated coefficients show that each unit increase in these predictors changes the odds of heart disease by the given multiplier. For example, a one-unit increase in the number of vessels colored (`ca`) increases the odds of heart disease by about `2.65` times.

Finally, we found that the test metrics show that the model performs slightly better on the test set than on the training set. Specifically, test accuracy increased to `86.81%` from `85.85%` in training, indicating a slight improvement in overall predictive performance. However, specificity decreased from `90.43%` to `85.71%`, suggesting the model is less effective at identifying non-cases of heart disease in the test set. Conversely, sensitivity rose significantly from `80.41%` to `88.1%`, showing a major improvement in correctly identifying cases. Overall, these results suggest that the model generalizes well to new data without overfitting.